In [2]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import os
from openai import OpenAI
import pandas as pd
import nltk

from sklearn.utils import shuffle
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from tqdm import tqdm

nltk.download('punkt')
nltk.download('wordnet')

C:\Users\Swifte\AppData\Local\Temp\ipykernel_39328\2640470385.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Swifte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Swifte\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [6]:
# !pip -q install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# auth.authenticate_user()  #click allow access
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [4]:
# Add openAI key

In [8]:

# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('balanced_test.csv')  

In [5]:
train = pd.read_csv('fulltrain.csv', header = None)
train.columns = ['label', 'text']
train

,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...
...,...,...
48849,4,The ruling Kuomintang (KMT) has claimed owners...
48850,4,The Taipei city government has encouraged the ...
48851,4,President Ma Ying-jeou said Friday that a park...
48852,4,The families of the four people who were kille...


In [6]:
test = pd.read_csv('balancedtest.csv', header = None)
test.columns = ['label', 'text']
test

,label,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...
...,...,...
2995,4,The Air Force mistakenly gave rival companies ...
2996,4,The United Nations climate chief on Friday cha...
2997,4,River Plate midfielder Diego Buonanotte has un...
2998,4,Lawmakers were on the brink Tuesday of exempti...


In [13]:
example_satire = train[(train['label'] == 1) & (train['text'].str.len() < 100)].sample(1)['text'].values[0]
example_hoax = train[(train['label'] == 2) & (train['text'].str.len() < 100)].sample(1)['text'].values[0]
example_propaganda = train[(train['label'] == 3) & (train['text'].str.len() < 100)].sample(1)['text'].values[0]
example_reliable = train[(train['label'] == 4) & (train['text'].str.len() < 100)].sample(1)['text'].values[0]
print(example_satire)
print(example_hoax)
print(example_propaganda)
print(example_reliable)



tools = [




    {




        "type": "function",




        "function": {




            "name": "classify_text",




            "description": f"Decide whether this news is a satire, hoax, propaganda, or reliable news. Use the following examples as a guide: satire: {example_satire}, hoax: {example_hoax}, propaganda: {example_propaganda}, reliable: {example_reliable}.",




            "parameters": {




                "type": "object",




                "required": ["news_type"],




                "properties": {




                    "news_type": {




                        "enum": ["satire", "hoax", "propaganda", "reliable"],




                        "type": "string",




                        "description": "Type of news",




                    },




                },




            },




        },




    }

]

FEMA has just released the following advisory for residents in ord near the deadly BBQ cloud: 
The Reason The Obama Administration Is Suing PA State Police Will Make You Angry 
A Brave New 1984Youtube 
Netherlands beat Brazil 2-1 in World Cup quarter-finals 


In [14]:
import asyncio

model_id = "gpt-3.5-turbo"



async def get_gpt_reply(m):
    loop = asyncio.get_event_loop()


    response = await loop.run_in_executor(
        None,
        lambda: openai.chat.completions.create(
            model=model_id,
            messages=m,
            tools=tools,
            tool_choice={"type": "function", "function": {"name": "classify_text"}},
        ),
    )



    return {
        "role": response.choices[0].message.role,
        "content": list(
            map(
                lambda x: x.function.arguments,
                response.choices[0].message.tool_calls,
            )
        )[0],
    }


messages = []

In [15]:
import nest_asyncio
import asyncio

async def do_nothing():
    return "NaN"

async def main(i: int):

    responses = list()
    pbar = tqdm(total=50)
    for _, row in test.iloc[i * 60 : (i + 1) * 60].iterrows():
        this_message = []

        prompt = row["text"]
        this_message.append({"role": "user", "content": prompt})
        try:
            output = get_gpt_reply(this_message)
            responses.append(output["content"].strip())
        except Exception:
            responses.append(do_nothing())
        pbar.update()

    pbar.close()
    results = await asyncio.gather(*responses)
    responses = []
    for result in results:
        responses.append(result)
    return responses


nest_asyncio.apply()

a = []
for i in tqdm(range(60)):
    a.extend(asyncio.run(main(i)))

  0%|          | 0/60 [00:00<?, ?it/s]C:\Users\Swifte\AppData\Local\Temp\ipykernel_39328\3109750331.py:17: RuntimeWarning: coroutine 'get_gpt_reply' was never awaited
  output = get_gpt_reply(this_message)
60it [00:00, 10000.33it/s]            
c:\Users\Swifte\anaconda3\envs\CS4248\lib\asyncio\tasks.py:232: RuntimeWarning: coroutine 'get_gpt_reply' was never awaited
  result = coro.send(None)
60it [00:00, 20009.40it/s]            
60it [00:00, 15003.77it/s]            
60it [00:00, 19162.28it/s]            
60it [00:00, 17011.98it/s]            
60it [00:00, 11972.32it/s]            
60it [00:00, 17041.93it/s]            
60it [00:00, 13289.23it/s]            
60it [00:00, 15003.77it/s]            
60it [00:00, 14999.30it/s]            
60it [00:00, 11783.41it/s]            
60it [00:00, 14583.81it/s]            
60it [00:00, 12004.88it/s]            15.45it/s]
60it [00:00, 23935.54it/s]            
60it [00:00, 20023.73it/s]            
60it [00:00, 19990.33it/s]            
60it [00:

In [16]:
model_id = "gpt-3.5-turbo"


def get_gpt_reply(m):

    response = openai.chat.completions.create(
        model=model_id,
        messages=m,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "classify_text"}},
    )

    return {
        "role": response.choices[0].message.role,
        "content": list(
            map(
                lambda x: x.function.arguments,
                response.choices[0].message.tool_calls,
            )
        )[0],
    }


messages = []

def main():

    responses = []
    pbar = tqdm(total=3000)
    for _, row in test.iterrows():
        this_message = []

        prompt = row["text"]
        this_message.append({"role": "user", "content": prompt})
        try:
            output = get_gpt_reply(this_message)
            responses.append(output)
        except Exception:
            responses.append("NaN")
        pbar.update()

    pbar.close()
    results = responses
    responses = []
    for result in results:
        print(result["content"].strip())
        responses.append(result["content"].strip())
    return responses

responses = main()

100%|██████████| 3000/3000 [40:58<00:00,  1.22it/s]  

{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"propaganda"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"propaganda"}
{"news_type":"hoax"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"propaganda"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"propaganda"}
{"news_type":"propaganda"}
{"news_type":"hoax"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"satire"}
{"news_type":"hoax"}
{"news_type":"satire"}
{"news_type":"satire

In [17]:
import json

def get_pred(response):
    match response:
        case "satire":
            return 1
        case "hoax":
            return 2
        case "propaganda":
            return 3
        case "reliable":
            return 4
        case _:
            return 0

responses = list(
    map(lambda response: get_pred(json.loads(response)["news_type"]), responses)
)
test["responses"] = responses

In [18]:
test.to_csv('result_test_oneshot.csv', encoding = 'utf-8-sig') 

In [20]:
results = pd.read_csv('result_test_oneshot.csv')
results = results[['text', 'label', 'responses']]
results

,text,label,responses
0,When so many actors seem content to churn out ...,1,1
1,In what football insiders are calling an unex...,1,1
2,In a freak accident following Game 3 of the N....,1,1
3,North Koreas official news agency announced to...,1,1
4,The former Alaska Governor Sarah Palin would b...,1,1
...,...,...,...
2995,The Air Force mistakenly gave rival companies ...,4,4
2996,The United Nations climate chief on Friday cha...,4,4
2997,River Plate midfielder Diego Buonanotte has un...,4,4
2998,Lawmakers were on the brink Tuesday of exempti...,4,4


In [21]:
from sklearn.metrics import classification_report

In [24]:
print(classification_report(results['label'], results['responses']))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.92      0.77      0.84       750
           2       0.59      0.23      0.33       750
           3       0.28      0.23      0.25       750
           4       0.50      0.96      0.65       750

    accuracy                           0.55      3000
   macro avg       0.46      0.44      0.41      3000
weighted avg       0.57      0.55      0.52      3000



C:\Users\Swifte\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Swifte\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Swifte\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
na_indices = results[results['raw_pred'].isna()].index

In [24]:
na_indices

Int64Index([756, 822, 854, 1179, 1301, 1379, 1865, 2015, 2079, 2216, 2257,
            2471, 2707],
           dtype='int64')

In [21]:
model_id = 'gpt-3.5-turbo'

def get_gpt_reply(m):
    response = openai.chat.completions.create(
        model=model_id,
        messages=m
    )
    m.append(
        {
            "role": response.choices[0].message.role,
            "content": list(
                map(
                    lambda x: x.function.arguments,
                    response.choices[0].message.tool_calls,
                )
            ),
        }
    )
    return m

messages = []
messages.append({'role': 'system', 'content': 'How may I help you?'})
messages = get_gpt_reply(messages)
print('{0}: {1}\n'.format(messages[-1]['role'].strip(), messages[-1]['content'].strip()))

assistant: Hello! As an AI language model, I can assist you with a variety of tasks. You can ask me for information, help with a project, personal advice, or just have a casual conversation. So, how can I be of assistance?



In [27]:
na_results = results.iloc[na_indices]
na_results

,text,label,raw_pred
756,Freed Slaves In Washington? Dont Blame Us We W...,2,NaN
822,"Mom complains about Bibles in her sons school,...",2,NaN
854,"Rehana, The Famous Female Kurdish Fighter Resp...",2,NaN
1179,11 Year Old Boy Being Changed Into Girl In Cal...,2,NaN
1301,What Liberals Demand Parents Do This Christmas...,2,NaN
1379,"Michael Browns Mother, Grandmother And Relativ...",2,NaN
1865,With a massive backlash now mounting against ...,3,NaN
2015,Jared Diamond is no doom-and-gloomer; he's a ...,3,NaN
2079,Prepare to have your world rocked. What you'r...,3,NaN
2216,U.S. Senator Charles Grassley of Iowa has con...,3,NaN


In [41]:
def reduce_text(text):
    tokens = word_tokenize(text)
    reduced_tokens = tokens[:3000]
    return ' '.join(reduced_tokens)

In [39]:
for index, row in na_results.iterrows():
    print(len(word_tokenize(reduce_text(row['text']))))

3500
3501
3500
3500
3500
3500
3501
1942
3500
682
3500
3500
991


In [42]:

na_responses = list()
pbar = tqdm(total=13)

for index, row in na_results.iterrows():
    this_message = []

    prompt = reduce_text(row['text'])
    this_message.append({'role': 'user', 'content': prompt})
    
    output = get_gpt_reply(this_message)
    na_responses.append(output[-1]['content'].strip())
    
    pbar.update()

pbar.close()

100%|███████████████████████████████████████████| 13/13 [00:34<00:00,  2.68s/it]


In [64]:
results.at[na_indices, 'raw_pred'] = na_responses

In [90]:
def get_pred(text):
    lower_text = text.lower()
    tokens = list(map(lambda t: stemmer.stem(t), word_tokenize(lower_text)))
    pred = list()
    if 'satire' in lower_text or stemmer.stem('satire') in tokens:
        pred.append(1)
    if 'hoax' in lower_text or stemmer.stem('hoax') in tokens:
        pred.append(2)
    if 'propaganda' in lower_text or stemmer.stem('propaganda') in tokens:
        pred.append(3)
    if ('reliable' in lower_text or stemmer.stem('reliable') in tokens) and 'unreliable' not in lower_text:
        pred.append(4)
    
    if len(pred) != 1:
        return 'NaN'
    return pred[0]

In [91]:
results['pred'] = results['raw_pred'].apply(get_pred)

In [110]:
cleaned_results = results[results['pred'] != 'NaN']
cleaned_results

,text,label,raw_pred,pred
0,When so many actors seem content to churn out ...,1,Satire.,1
1,In what football insiders are calling an unex...,1,Satire.,1
2,In a freak accident following Game 3 of the N....,1,The news article appears to be a satire.,1
3,North Koreas official news agency announced to...,1,Satire.,1
4,The former Alaska Governor Sarah Palin would b...,1,Satire.,1
...,...,...,...,...
2995,The Air Force mistakenly gave rival companies ...,4,This is a reliable news article.,4
2996,The United Nations climate chief on Friday cha...,4,Reliable News.,4
2997,River Plate midfielder Diego Buonanotte has un...,4,This is a reliable news.,4
2998,Lawmakers were on the brink Tuesday of exempti...,4,This news appears to be reliable news.,4


In [87]:
stemmer.stem('satirical')

'satir'

In [88]:
stemmer.stem('satire')

'satir'

In [111]:
cleaned_results.to_csv('result_test.csv', index=False)

## Results analysis

In [113]:
test_pred = pd.read_csv('result_test.csv')
test_pred

,text,label,raw_pred,pred
0,When so many actors seem content to churn out ...,1,Satire.,1
1,In what football insiders are calling an unex...,1,Satire.,1
2,In a freak accident following Game 3 of the N....,1,The news article appears to be a satire.,1
3,North Koreas official news agency announced to...,1,Satire.,1
4,The former Alaska Governor Sarah Palin would b...,1,Satire.,1
...,...,...,...,...
2867,The Air Force mistakenly gave rival companies ...,4,This is a reliable news article.,4
2868,The United Nations climate chief on Friday cha...,4,Reliable News.,4
2869,River Plate midfielder Diego Buonanotte has un...,4,This is a reliable news.,4
2870,Lawmakers were on the brink Tuesday of exempti...,4,This news appears to be reliable news.,4


In [11]:
from sklearn.metrics import classification_report

In [126]:
print(classification_report(test_pred['label'], test_pred['pred']))

              precision    recall  f1-score   support

           1       0.88      0.98      0.93       749
           2       0.73      0.02      0.03       697
           3       0.37      0.40      0.39       705
           4       0.56      0.99      0.72       721

    accuracy                           0.61      2872
   macro avg       0.64      0.60      0.52      2872
weighted avg       0.64      0.61      0.52      2872



In [9]:
df = pd.read_csv('result_test_raw_gpt42.csv')
df

,Unnamed: 0,label,text,responses
0,0,1,When so many actors seem content to churn out ...,1
1,1,1,In what football insiders are calling an unex...,1
2,2,1,In a freak accident following Game 3 of the N....,1
3,3,1,North Koreas official news agency announced to...,2
4,4,1,The former Alaska Governor Sarah Palin would b...,1
...,...,...,...,...
2995,2995,4,The Air Force mistakenly gave rival companies ...,4
2996,2996,4,The United Nations climate chief on Friday cha...,4
2997,2997,4,River Plate midfielder Diego Buonanotte has un...,4
2998,2998,4,Lawmakers were on the brink Tuesday of exempti...,4


In [12]:
print(classification_report(df['label'], df['responses']))

              precision    recall  f1-score   support

           1       0.85      0.89      0.87       750
           2       0.37      0.11      0.16       750
           3       0.34      0.20      0.25       750
           4       0.47      0.98      0.64       750

    accuracy                           0.54      3000
   macro avg       0.51      0.54      0.48      3000
weighted avg       0.51      0.54      0.48      3000



In [27]:
df = pd.read_csv('result_test_raw.csv')
df


,Unnamed: 0,label,text,responses
0,0,1,When so many actors seem content to churn out ...,"{""news_type"":""satire""}"
1,1,1,In what football insiders are calling an unex...,"{""news_type"":""satire""}"
2,2,1,In a freak accident following Game 3 of the N....,"{""news_type"":""satire""}"
3,3,1,North Koreas official news agency announced to...,"{""news_type"":""satire""}"
4,4,1,The former Alaska Governor Sarah Palin would b...,"{""news_type"":""reliable""}"
...,...,...,...,...
2995,2995,4,The Air Force mistakenly gave rival companies ...,"{""news_type"":""reliable""}"
2996,2996,4,The United Nations climate chief on Friday cha...,"{""news_type"":""reliable""}"
2997,2997,4,River Plate midfielder Diego Buonanotte has un...,"{""news_type"":""reliable""}"
2998,2998,4,Lawmakers were on the brink Tuesday of exempti...,"{""news_type"":""reliable""}"


In [40]:
def get_pred(response):
    response = json.loads(response)
    print(response)
    match response['news_type']:
        case "satire":
            return 1
        case "hoax":
            return 2
        case "propaganda":
            return 3
        case "reliable":
            return 4
        case _:
            return 0

In [41]:
df['pred'] = df['responses'].apply(get_pred)
df

{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'reliable'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'propaganda'}
{'news_type': 'satire'}
{'news_type': 'propaganda'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'hoax'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'hoax'}
{'news_type': 'hoax'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'reliable'}
{'news_type': 'propaganda'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'hoax'}
{'news_type': 'hoax'}
{'news_type': 'satire'}
{'news_type': 'propaganda'}
{'news_type': 'hoax'}
{'news_type': 'propaganda'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'satire'}
{'news_type': 'hoax'}
{'news

,Unnamed: 0,label,text,responses,pred
0,0,1,When so many actors seem content to churn out ...,"{""news_type"":""satire""}",1
1,1,1,In what football insiders are calling an unex...,"{""news_type"":""satire""}",1
2,2,1,In a freak accident following Game 3 of the N....,"{""news_type"":""satire""}",1
3,3,1,North Koreas official news agency announced to...,"{""news_type"":""satire""}",1
4,4,1,The former Alaska Governor Sarah Palin would b...,"{""news_type"":""reliable""}",4
...,...,...,...,...,...
2995,2995,4,The Air Force mistakenly gave rival companies ...,"{""news_type"":""reliable""}",4
2996,2996,4,The United Nations climate chief on Friday cha...,"{""news_type"":""reliable""}",4
2997,2997,4,River Plate midfielder Diego Buonanotte has un...,"{""news_type"":""reliable""}",4
2998,2998,4,Lawmakers were on the brink Tuesday of exempti...,"{""news_type"":""reliable""}",4


In [42]:
print(classification_report(df['label'], df['pred']))

              precision    recall  f1-score   support

           1       0.90      0.68      0.78       750
           2       0.57      0.35      0.43       750
           3       0.28      0.16      0.20       750
           4       0.47      0.96      0.63       750

    accuracy                           0.54      3000
   macro avg       0.55      0.54      0.51      3000
weighted avg       0.55      0.54      0.51      3000



: 